In [ ]:
import pandas as pd
import scipy
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import lib.helpers as hlp
import lib.backtest as backtest
import lib.plotting as plotting
import lib.kelly as kelly
import lib.distributions as dst
import lib.random_path_generation as rpg
import lib.convolution as convolution



In [ ]:
# all backtest settings
historical_df = hlp.get_historical_prices_df()
price_sequence = historical_df['Bitcoin'].dropna()
returns_sequence = price_sequence.pct_change()[1:]
simulation_length_days = 1000 # number of days to use for backtest
strike = 1. # 1.1 = 10% in the money
duration = 5 # number of the expiration days for put option
number_of_paths = 1000 # number of the random paths to do backtest
bonding_curve_resolution = 100 # number of utils on the [0, 1] segment, passed to the get_kelly_curve
n_bins = np.linspace(-1, 1, 1000) # Number of bins. Could be also string (e.g. 'auto') or list of bin edges. Whatever complies with np.histogram
mean_shifts = np.arange(-0.002, 0.003, 0.001)*100 ##ADDING FOR INTENSITY
premium_offset = 0. # 0.1 = 10% additional relative premium
# n_bins = 'auto'
utilizations = [float(i/10) for i in range(0,11)] # different utilizations to try out
convolution_n_paths = 10000 # number of paths to use in Monte Carlo Convolution

original_returns_histogram_df = dst.get_returns_histogram_from_returns_sequence(returns_sequence, n_bins=n_bins)
returns_std = returns_sequence.std()

option_type = 'put'

In [ ]:
print(scipy.stats.skew(returns_sequence.dropna()))

In [ ]:
# create the kelly curve
convolved_returns_histogram_df = convolution.monte_carlo_convolution(
    returns_sequence,
    n_bins=n_bins,
    number_of_paths=convolution_n_paths,
    n_convolutions=duration,
)
underlying_n_daily_returns = convolution.daily_returns_to_n_day_returns(returns_sequence, 
                                                                        number_of_paths=convolution_n_paths, 
                                                                        n_days=duration)
random_return_paths_cycles_df = rpg.generate_returns_paths_from_returns(
    underlying_n_daily_returns, number_of_paths, simulation_length_days//duration, output_as_df=True
)

kelly_curve_df = kelly.get_kelly_curve(
    underlying_n_daily_returns, strike, number_of_utils=bonding_curve_resolution, option_type=option_type)

fit_params = kelly.fit_curve_parameters(
    kelly_curve_df["util"], kelly_curve_df["premium"]
)

In [ ]:
# Widget which allows to contol the Bonding Curve
# Run cell => set appropriate values => run cells below
%matplotlib notebook
a, b, c, d = fit_params
a_slider = widgets.FloatSlider(
    value=a,
    min=0,
    max=1,
    step=0.01,   
)
b_slider = widgets.FloatSlider(
    value=b,
    min=0,
    max=20,
    step=0.01,
    
)
c_slider = widgets.FloatSlider(
    value=c,
    min=-1,
    max=100,
    step=0.01,
    
)
d_slider = widgets.FloatSlider(
    value=d,
    min=0,
    max=1,
    step=0.01,
    
)

def plot_bonding_curve(a=a, b=b, c=c, d=d):
    "showing sine frequency"
    f, ax1 = plt.subplots(nrows=1,figsize=(8,6))
    curve_df = kelly.get_curve_df(a, b, c, d, n_samples=100)
    ax1.plot(curve_df["util"], curve_df["premium"], 'b')
    ax1.set_ylim(ymin=0, ymax=1.1)


widgets.interactive(plot_bonding_curve, a=a_slider, 
                                        b=b_slider, 
                                        c=c_slider, 
                                        d=d_slider)


In [ ]:
# Widget with Kelly Curve Function
def print_curve_params(a, b, c, d):
    print(f'{a} * x * cosh({b} * x ** {c}) + {d}')
          
out = widgets.interactive_output(print_curve_params, {'a': a_slider, 'b': b_slider, 'c': c_slider, 'd': d_slider})

widgets.HBox([out])

In [ ]:
assets_info_dicts = []
for mean_shift in tqdm(mean_shifts):
    
    tmp_underlying_n_daily_returns = underlying_n_daily_returns + mean_shift
    random_return_paths_cycles_df = rpg.generate_returns_paths_from_returns(
        tmp_underlying_n_daily_returns, number_of_paths, simulation_length_days//duration, output_as_df=True
    ) 
    for util in utilizations:
        asset_dict = {}
        asset_dict['price_sequence'] = price_sequence
        asset_dict['returns'] = returns_sequence
        asset_dict['mean_shift'] = mean_shift
        asset_dict['util'] = util
        asset_dict['shift'] = mean_shift
        asset_dict['utils'] = rpg.generate_utils_list(simulation_length_days, initial_util=asset_dict['util'], 
                                                      duration=duration, randomize=False)
        asset_dict['strike'] = strike
        asset_dict['duration'] = duration
        asset_dict['years'] = simulation_length_days / 365
        
        
        asset_dict['returns_histgram_df'] = \
            dst.get_returns_histogram_from_returns_sequence(tmp_underlying_n_daily_returns, n_bins=n_bins)
        
        
        premiums, _, _ = kelly.get_premiums_list_with_all_calculations(
            asset_dict['utils'],
            premium_offset=premium_offset,
            fit_params=[a_slider.value, b_slider.value, c_slider.value, d_slider.value]
        )
        
        bankroll_df = backtest.run_backtest(
            random_return_paths_cycles_df,
            asset_dict['utils'],
            premiums,
            strike,
            duration, 
            option_type=option_type
        )
 
        asset_dict["max_drawdown_percentiles"] = backtest.calculate_percentiles(
            backtest.calculate_max_drawdown,
            bankroll_df,
            percentiles=(0, 25, 50, 75, 100),
        )
        asset_dict["cagr_percentiles"] = backtest.calculate_percentiles(
            backtest.calculate_cagr, bankroll_df, percentiles=(0, 25, 50, 75, 100)
        )
        asset_dict["sharpe_percentiles"] = backtest.calculate_percentiles(
            backtest.calculate_sharpe, bankroll_df, percentiles=(0, 25, 50, 75, 100)
        )
        
        asset_dict['label'] = f"util={util}_shift={mean_shift}"
        assets_info_dicts.append(asset_dict)

backtest_results = pd.DataFrame()
backtest_results['util'] = utilizations
backtest_results['final_bankroll'] = [asset_dict["cagr_percentiles"][2] for asset_dict in assets_info_dicts if asset_dict['mean_shift']==0]

In [ ]:
# plot one day returns distributions
fig = go.Figure()
for asset_dict in assets_info_dicts:
    
     fig.add_trace(go.Scatter(x=asset_dict['returns_histgram_df']['return'], 
                              y=asset_dict['returns_histgram_df']['freq'],
                              mode='lines',
                              name=asset_dict['label']))
    
fig.update_xaxes(range=[-1,1])
fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis_title='Return (%)',
        yaxis_title='Probability Density').update_xaxes(showgrid=False).update_yaxes(showgrid=False)

fig.show()

In [ ]:
# convert list of dicts to the summary
summary_df = pd.DataFrame(assets_info_dicts)
max_drawdown_percentile_df = pd.DataFrame(
    summary_df.max_drawdown_percentiles.tolist(),
    index=summary_df.index,
)
max_drawdown_percentile_df.rename(
    columns=lambda col: f"{(0, 25, 50, 75, 100)[col]}_percentile_max_drawdown",
    inplace=True,
)
cagr_percentile_df = pd.DataFrame(
    summary_df.cagr_percentiles.tolist(), index=summary_df.index
)
cagr_percentile_df.rename(
    columns=lambda col: f"{(0, 25, 50, 75, 100)[col]}_percentile_cagr",
    inplace=True,
)

sharpe_percentile_df = pd.DataFrame(
    summary_df.sharpe_percentiles.tolist(), index=summary_df.index
)
sharpe_percentile_df.rename(
    columns=lambda col: f"{(0, 25, 50, 75, 100)[col]}_percentile_sharpe",
    inplace=True,
)

summary_df = summary_df.merge(
    max_drawdown_percentile_df, left_index=True, right_index=True
)
summary_df = summary_df.merge(
    cagr_percentile_df, left_index=True, right_index=True
)
summary_df = summary_df.merge(
    sharpe_percentile_df, left_index=True, right_index=True
)
(summary_df[['util',  'shift', '50_percentile_cagr', '50_percentile_max_drawdown']]).head(5)

In [ ]:
# calculate drawdowns and cagr heatmaps
max_drawdown_df = pd.DataFrame(index=summary_df['util'].unique().round(1), columns=sorted(summary_df['shift'].unique()))
cagr_df = pd.DataFrame(index=summary_df['util'].unique().round(1), columns=sorted(summary_df['shift'].unique()))

for index, row in summary_df.iterrows():
    max_drawdown_df.at[round(row['util'], 1), row['shift']] = row['50_percentile_max_drawdown']
    cagr_df.at[round(row['util'], 1), row['shift']] = row['50_percentile_cagr']

In [ ]:
%matplotlib inline
# Max Drawdown Heatmap
sns.color_palette("dark:salmon_r", as_cmap=True)
sns.heatmap(max_drawdown_df.sort_index(ascending=False).astype(float), annot=False,vmin=0,vmax=1)
plt.title('Max Drawdown Heatmap')
plt.xlabel("Sigma Distribution Shift")
plt.ylabel("Utilization") 
plt.show()

In [ ]:
# Median Cagr Heatmap
sns.color_palette("coolwarm", as_cmap=True)
cmap = sns.diverging_palette(20, 220, as_cmap=True)
sns.heatmap(cagr_df.astype(float), annot=False, center=0, vmin=-1, vmax=1, cmap=cmap)
plt.title('Median CAGR')
plt.xlabel("Sigma Distribution Shift")
plt.ylabel("Utilization") 
plt.show()

In [ ]:
sns.kdeplot(
    data=summary_df, x='50_percentile_max_drawdown', y='50_percentile_cagr', fill=True
)

In [ ]:
# Test backtest bankrolls vs Analytical

In [ ]:
# analytical Kelly returns calculation
analytical_kelly_returns = []

for util, premium in zip(kelly_curve_df['util'], kelly_curve_df['premium']):
    # log_return = kelly.get_kelly_log_expected_payout(returns_histogram_df, strike, p, u)
    log_return = kelly.get_kelly_log_expected_payout(underlying_n_daily_returns, strike, premium, util, option_type=option_type)
    analytical_kelly_returns.append(np.power(np.exp(log_return), simulation_length_days//duration))

    #expected_gain = math.exp(iterations*(odds*math.log(1+f)+(1-odds)*math.log(1-f)))
analytical_kelly_returns = [hlp.convert_bankroll_to_cagr(bankroll, simulation_length_days/365) for bankroll in analytical_kelly_returns]


In [ ]:
# plot both
fig = go.Figure()
fig.add_trace(go.Scatter(x=kelly_curve_df['util'], y=analytical_kelly_returns, name='analytical'))
fig.add_trace(go.Scatter(x=backtest_results['util'], y=backtest_results['final_bankroll'], name='backtested'))